Train CVE description with no subject

In [5]:
import pickle
import numpy as np
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
from keras.callbacks import Callback
from sklearn.preprocessing import LabelEncoder

class F1ScoreCallback(Callback):
    def __init__(self, X_val, y_val):
        super(F1ScoreCallback, self).__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0
        self.best_model = None
        self.f1_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_val_pred = np.argmax(self.model.predict(self.X_val), axis=1)
        f1 = f1_score(self.y_val, y_val_pred, average='weighted')
        self.f1_scores.append(f1)
        

        if f1 > self.best_f1:
            self.best_f1 = f1
            self.best_model = self.model
            print(f"Epoch {epoch + 1} - F1 Score: {f1:.4f}")
            print("Saved best model")
            print(self.f1_scores)

with open('train_no_subj.pickle', 'rb') as f1:
    balanced = pickle.load(f1)

with open('test_no_subj.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

train = np.array([item['cve_description_no_subject_ada_embedding'] for item in balanced if item['cwe'] != 'None'])
test = np.array([item['cwe'] for item in balanced if item['cwe'] != 'None'])
np.random.seed(42)
X_train, X_val, y_train, y_val = train_test_split(train,test,test_size=0.1,random_state=42)

X_test = np.array([item['cve_description_no_subject_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

label_encoder_train = LabelEncoder()
y_train_encoded = label_encoder_train.fit_transform(y_train)
label_encoder_test = LabelEncoder()
y_test_encoded = label_encoder_test.fit_transform(y_test)


input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))

model = Sequential()
model.add(Dense(128, input_dim=input_dim, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

f1_callback = F1ScoreCallback(X_val, label_encoder_train.transform(y_val))

history = model.fit(X_train, y_train_encoded, epochs=40, batch_size=32, validation_data=(X_val, label_encoder_train.transform(y_val)), verbose=1, callbacks=[f1_callback])

best_model = f1_callback.best_model


# Save the best model
joblib.dump(best_model, 'CWE_classes.joblib')

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

joblib.dump(label_encoder_train, 'label_encoder_train.joblib')

Epoch 1/40
227/227 [==============================] - 1s 3ms/step loss: 
Epoch 1 - F1 Score: 0.5412
Saved best model
[0.5412277425555728]
2036/2036 [==============================] - 11s 5ms/step - loss: 2.0307 - accuracy: 0.4674 - val_loss: 1.5372 - val_accuracy: 0.5747
Epoch 2/40
227/227 [==============================] - 1s 3ms/step loss: 1
Epoch 2 - F1 Score: 0.5964
Saved best model
[0.5412277425555728, 0.5963509858941246]
2036/2036 [==============================] - 11s 6ms/step - loss: 1.4063 - accuracy: 0.6073 - val_loss: 1.3792 - val_accuracy: 0.6161
Epoch 3/40
227/227 [==============================] - 1s 3ms/step loss:
Epoch 3 - F1 Score: 0.6091
Saved best model
[0.5412277425555728, 0.5963509858941246, 0.6090936473846966]
2036/2036 [==============================] - 10s 5ms/step - loss: 1.2803 - accuracy: 0.6401 - val_loss: 1.3366 - val_accuracy: 0.6252
Epoch 4/40
227/227 [==============================] - 0s 1ms/step loss: 1.2108 - accura
Epoch 4 - F1 Score: 0.6295
Saved bes

['label_encoder_train.joblib']

Inference CVE description with no subject

In [8]:
import pickle
import numpy as np
from sklearn.metrics import classification_report
import joblib

# Load the saved model
best_model = joblib.load('CWE_classes.joblib')

# Load the label encoder
label_encoder_train = joblib.load('label_encoder_train.joblib')

# Load the test data
with open('test_no_subj.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

X_test = np.array([item['cve_description_no_subject_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Convert the predicted labels back to their original form
y_pred_original = label_encoder_train.inverse_transform(y_pred)

# Generate and print the classification report
print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))


402/402 [==============================] - 0s 997us/step
Classification Report:
               precision    recall  f1-score   support

         119     0.8691    0.3785    0.5273      1070
         120     0.2634    0.6531    0.3754       196
         125     0.6765    0.8214    0.7419       532
         134     0.7857    0.5789    0.6667        19
         190     0.5630    0.6700    0.6119       200
          20     0.4936    0.2877    0.3635       810
         200     0.7359    0.4864    0.5857       590
         203     0.4524    0.7037    0.5507        27
          22     0.7496    0.8147    0.7808       518
         269     0.3646    0.3302    0.3465       106
         276     0.1651    0.2812    0.2081        64
         287     0.3477    0.6526    0.4537       285
         295     0.5684    0.6667    0.6136        81
         306     0.3457    0.2979    0.3200        94
         312     0.2857    0.2381    0.2597        42
         319     0.4600    0.4510    0.4554        51
